In [8]:
# importing pandas
import pandas as pd
#fetch data and add headers
filepath = 'promise_CM1.csv'
column = ["loc","v(g)","ev(g)","n","v","l","d","i","e","b","t",
          "loCode","loComment","loBlank","locCodeAndComment",
          "uniq_Opnd","total_Op","branchCount","defects"]
df = pd.read_csv(filepath, names=column)

In [9]:
X = df.iloc[:,0:-1] #features
y = df.iloc[:,-1] #labels

In [10]:
#divide dataset into train and test
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# model for Gausian NB

gnb = GaussianNB().fit(X_train, y_train)

#predict model on test data
#predicted = gnb.predict(X_test)

In [16]:
#exhaustive filtering using K nearest neighbour
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS



efs1 = EFS(gnb, 
           min_features=4,
           max_features=18,
           scoring='accuracy',
           print_progress=True,
           cv=5)

efs1 = efs1.fit(X, y)

#print('Best accuracy score: %.2f' % efs1.best_score_)
print('Best subset (indices):', efs1.best_idx_)
print('Best subset (corresponding names):', efs1.best_feature_names_)

Features: 261156/261156

Best accuracy score: 0.90
Best subset (indices): (3, 6, 7, 10)
Best subset (corresponding names): ('n', 'd', 'i', 't')


In [17]:
X_train_efs = efs1.transform(X_train)
X_test_efs = efs1.transform(X_test)

gnb.fit(X_train_efs,y_train)
y_pred = gnb.predict(X_test_efs)

acc = float((y_test == y_pred).sum()) / y_pred.shape[0]
print('Test set accuracy: %.2f %%' % (acc*100))

Test set accuracy: 86.67 %


In [13]:
df = pd.DataFrame.from_dict(efs1.get_metric_dict()).T
df.sort_values('avg_score',inplace=True, ascending = False)
df

,feature_idx,cv_scores,avg_score,feature_names,ci_bound,std_dev,std_err
705,"(8, 10, 12)","[0.9, 0.92, 0.91, 0.88, 0.9081632653061225]",0.903633,"(e, t, loComment)",0.0172503,0.0134213,0.00671066
624,"(6, 10, 12)","[0.9, 0.92, 0.91, 0.88, 0.9081632653061225]",0.903633,"(d, t, loComment)",0.0172503,0.0134213,0.00671066
598,"(6, 7, 10)","[0.9, 0.92, 0.91, 0.88, 0.9081632653061225]",0.903633,"(d, i, t)",0.0172503,0.0134213,0.00671066
9965,"(3, 6, 7, 10, 12)","[0.9, 0.92, 0.91, 0.88, 0.9081632653061225]",0.903633,"(n, d, i, t, loComment)",0.0172503,0.0134213,0.00671066
3399,"(6, 7, 10, 12)","[0.9, 0.92, 0.91, 0.88, 0.9081632653061225]",0.903633,"(d, i, t, loComment)",0.0172503,0.0134213,0.00671066
...,...,...,...,...,...,...,...
104472,"(4, 5, 9, 11, 13, 14, 15, 16)","[0.83, 0.85, 0.83, 0.76, 0.8673469387755102]",0.827469,"(v, l, b, loCode, loBlank, locCodeAndComment, ...",0.0469185,0.0365042,0.0182521
91000,"(1, 4, 5, 9, 11, 13, 14, 16)","[0.83, 0.85, 0.83, 0.76, 0.8673469387755102]",0.827469,"(v(g), v, l, b, loCode, loBlank, locCodeAndCom...",0.0469185,0.0365042,0.0182521
87704,"(1, 3, 4, 5, 9, 13, 15, 17)","[0.82, 0.84, 0.84, 0.77, 0.8673469387755102]",0.827469,"(v(g), n, v, l, b, loBlank, uniq_Opnd, branchC...",0.0416988,0.0324431,0.0162216
76265,"(0, 3, 4, 5, 9, 13, 16, 17)","[0.82, 0.83, 0.84, 0.77, 0.8673469387755102]",0.825469,"(loc, n, v, l, b, loBlank, total_Op, branchCount)",0.0410173,0.0319129,0.0159564
